In [2]:
import torch
kps = torch.rand(size = (3,200))
print(kps)

tensor([[0.7926, 0.8989, 0.7016, 0.1928, 0.3499, 0.5471, 0.0804, 0.5714, 0.6135,
         0.2638, 0.0998, 0.5471, 0.6459, 0.9834, 0.7999, 0.3662, 0.6372, 0.0341,
         0.5309, 0.2088, 0.7528, 0.0432, 0.1999, 0.4485, 0.2451, 0.5008, 0.2646,
         0.1830, 0.9369, 0.7591, 0.8726, 0.3076, 0.9854, 0.2189, 0.9423, 0.9552,
         0.7224, 0.6225, 0.4746, 0.3116, 0.6831, 0.2869, 0.5111, 0.3511, 0.9668,
         0.6949, 0.3008, 0.0087, 0.4909, 0.8517, 0.0437, 0.7792, 0.0029, 0.0066,
         0.9524, 0.5140, 0.8085, 0.0052, 0.1004, 0.9859, 0.8414, 0.8295, 0.5968,
         0.3847, 0.0407, 0.5694, 0.6962, 0.0443, 0.1782, 0.0308, 0.5170, 0.2857,
         0.5095, 0.0181, 0.1536, 0.6669, 0.9632, 0.0958, 0.6271, 0.9065, 0.2251,
         0.4262, 0.1498, 0.1442, 0.9660, 0.9635, 0.7472, 0.3926, 0.2883, 0.3503,
         0.7226, 0.3841, 0.8317, 0.5184, 0.5431, 0.6615, 0.4013, 0.9322, 0.7367,
         0.2054, 0.6911, 0.2419, 0.0066, 0.4634, 0.2009, 0.9546, 0.7582, 0.7070,
         0.7464, 0.3033, 0.0

In [3]:
import torch
import numpy as np


def generateRandomHomography(shape, GLOBAL_MULTIPLIER = 0.3):

    theta = np.radians(np.random.normal(0, 12.0*GLOBAL_MULTIPLIER)) # 生成随机旋转角度

    scale = np.random.normal(0, 1.1*GLOBAL_MULTIPLIER)#0.15*GLOBAL_MULTIPLIER) # 随机尺度
    if scale < 0.0: # get the right part of the gaussian
        scale = 1.0/(1. + abs(scale))
    else:
        scale = 1. + scale

    tx , ty = -shape[1]/2.0 , -shape[0]/2.0 
    txn, tyn = np.random.normal(0, 80.0*GLOBAL_MULTIPLIER, 2) #translation error 平移误差
    c, s = np.cos(theta), np.sin(theta)

    sx , sy = np.random.normal(0,0.6*GLOBAL_MULTIPLIER,2)
    p1 , p2 = np.random.normal(0,0.006*GLOBAL_MULTIPLIER,2)
    if np.random.uniform() > 0.5:
        scale_r = np.random.uniform(max(0.2, 0.8 - GLOBAL_MULTIPLIER), 1.)
    else:
        scale_r = np.random.uniform(1., min(4., 1.2 + 3.*GLOBAL_MULTIPLIER))

    H_t = np.array(((1,0, tx), (0, 1, ty), (0,0,1))) #t

    H_r = np.array(((c,-s, 0), (s, c, 0), (0,0,1))) #rotation,
    H_a = np.array(((1,sy, 0), (sx, 1, 0), (0,0,1))) # affine
    H_p = np.array(((1, 0, 0), (0 , 1, 0), (p1,p2,1))) # projective

    H_s = np.array(((scale,0, 0), (0, scale * scale_r, 0), (0,0,1))) #scale
    H_b = np.array(((1.0,0,-tx +txn), (0, 1, -ty + tyn), (0,0,1))) #t_back,

    #H = H_e * H_s * H_a * H_p
    H = np.dot(np.dot(np.dot(np.dot(np.dot(H_b,H_s),H_p),H_a),H_r),H_t)

    return H

H = torch.tensor([generateRandomHomography([300,240], 0.3) for b in range(16)],
                               dtype = torch.float32)

/tmp/ipykernel_26074/4167812401.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  H = torch.tensor([generateRandomHomography([300,240], 0.3) for b in range(16)],


In [5]:
generateRandomHomography([300,240], 0.3).shape

(3, 3)